In [ ]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import urllib
import pandas as pd
import re
import datetime
import numpy as np
import pickle
import requests
from time import sleep
import pandas as pd
import random
import os

In [ ]:
version = "1.0.0"
datadir = "data/"
archivedir = "data/Archive"

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import ui
from selenium.webdriver.support.ui import WebDriverWait, Select

from selenium.webdriver.firefox.options import Options
options = Options()
options.headless = True
driver = webdriver.Firefox(options=options)

In [ ]:
def gedjournalarticles(filename):
    # open old file
    olddf = pd.DataFrame({'url': [] })
    try:
        olddf = pd.read_pickle(os.path.join(archivedir, filename))
        print(olddf.tail())
    except:
        pass

    driver.get("http://periodicos.pucminas.br/index.php/abakos/issue/archive")
    
    # get all of the issue
    def issues():
        issues = []
        # get all of the Article on this page
        content = driver.page_source
        # load the page content in BeautifulSoup
        soup = BeautifulSoup(content, features="lxml")
        # get all of the issue div
        for div in soup.find_all('div', {'class': 'obj_issue_summary'}):
            for a in div.find_all('a', {'class': 'title'}):                
                issues.append(a['href'])        
        
        return  issues    
    allissue = issues()
    allissue = list(set(allissue))
    
    def articles():
        articles = []
        nexpage = False
        # get all of the Article on this page
        content = driver.page_source
        # load the page content in BeautifulSoup
        soup = BeautifulSoup(content, features="lxml")
        # We collect the texts under the Articles         
        # get the 'Artilces' h2
        for div in soup.find_all('div', {'class': 'obj_article_summary'}):
            for title in div.find_all('div', {'class': 'title'}):
                for a in title.find_all('a'):
                    articles.append(a['href'])
                    
            
        return  articles
    
    # iterate over isses and get article links
    articellinks = []
    for issue in allissue:
        driver.get(issue)
        sleep(2)
        print(driver.current_url, end='\r')         
        thispagearticles = articles()
        articellinks = articellinks + thispagearticles        
    articellinks = list(set(articellinks))
    
    # filter out old articles
    oldarticles = olddf['url']
    articellinks = list( set(articellinks).difference(set(oldarticles) ))
    print(len(articellinks), "new article found!")
    
    # iterate over article 
    urls = [ None for _ in range(len(articellinks))]
    titles = [ None for _ in range(len(articellinks))]
    abstracts = [ None for _ in range(len(articellinks))]
    writers = [ None for _ in range(len(articellinks))]
    dates = [ None for _ in range(len(articellinks))]
    dois = [  None for _ in range(len(articellinks)) ]
    keywords = [  None for _ in range(len(articellinks)) ]
    languages = [ None for _ in range(len(articellinks)) ]
    for idx in range(len(articellinks)):
        # print percentiage of the process
        print( str(np.round(100*idx/len(articellinks),2))+"%" , end='\r')
        url =  articellinks[idx]
        # load page
        driver.get(articellinks[idx])
        sleep(3)        
        urls[idx] = driver.current_url
        # get the issue article
        # get all of the Article on this page
        content = driver.page_source
        # load the page content in BeautifulSoup
        soup = BeautifulSoup(content, features="lxml")
        # process metadata
        keyword = []
        writter = []
        abstract = ""
        isarticle = True
        for m in soup.find_all("meta"):
            if m.has_attr("name"):
                if m['name'] == "DC.Type.articleType":
                    if not re.search('Full papers', m['content']) and re.search('Full papers', m['content']):
                        isarticle = False
                if m['name'] == "keywords":
                    keyword = [ x.strip() for x in m['content'].split(",") ]
                if m['name'] == "DC.Date.issued":
                    dates[idx] = datetime.datetime.strptime(m['content'], "%Y-%m-%d").strftime('%Y-%m-%d')
                if m['name'] == 'DC.Creator.PersonalName':
                    writter.append(m['content'])
                if m['name'] == 'citation_doi':
                    dois[idx] = m['content']
                if m['name'] == 'DC.Description':
                    abstracts[idx] = m['content']
                if m['name'] == 'citation_language':
                    languages[idx] = m['content']
                if m['name'] == 'DC.Title':
                     titles[idx] = m['content']
        # if it is not Article (like editor or Communication) we will drop
        if not isarticle:
            abstracts[idx] = None
        if len(writter) > 0:
            writers[idx] =  "#".join(writter)
        if len(keyword) > 0:
            keywords[idx] =  "#".join(keyword)
        # filter out DOI from title
        print(dois[idx],  titles[idx])
        if dois[idx] is not None and titles[idx] is not None:
            if re.search(dois[idx], titles[idx]):
                titles[idx] = titles[idx].replace("  ", " ")
                titles[idx] = titles[idx].replace(" DOI - "+dois[idx], '')
                titles[idx] = titles[idx].replace(" DOI - "+dois[idx].upper(), '')
                titles[idx] = titles[idx].replace(" - DOI "+dois[idx], '')
                titles[idx] = titles[idx].replace(" - DOI "+dois[idx].upper(), '')
                titles[idx] = titles[idx].replace(" DOI "+dois[idx], '')
                titles[idx] = titles[idx].replace(" DOI "+dois[idx].upper(), '')
                titles[idx] = titles[idx].replace(dois[idx], '')
    
    # make df
    df = pd.DataFrame({'url': urls, 
                       'journal_title': "Abakós", 
                       'journal_eissn': "2316-9451",
                       'journal_pissn': '',
                       'category': "Information technology",                                              
                      })
    # extend the df
    df['title'] = titles
    df['doi'] = dois
    df['abstract'] = abstracts
    df['writer'] = writers
    df['publishdate'] = dates
    df['keyword'] = keywords
    df['language'] = languages
    
    # drop where is no abstract
    df = df[ df['abstract'].notna()]
    # drop where title is Editorial
    df = df[df['title'] != 'Editorial']    
    
    # check there was a not Arhived but previously loaded file
    adf = None
    try:
        adf = pd.read_pickle(os.path.join(datadir, filename))
    except:
        pass
    if adf is not None:
        df = pd.concat([df, adf])
        df.drop_duplicates(inplace=True)
    
    return df, olddf

In [ ]:
filename = 'journal_Abakos_'+version+'.pandas'
    
# search for articles
df, olddf = gedjournalarticles( filename )

# save data
df.to_pickle(os.path.join(datadir, filename))

print(df.head())


In [ ]:
print(df.iloc[0]['abstract'])

In [ ]:
print(df.iloc[0])

In [ ]:
driver.close()

In [ ]:
print(df['title'].values)

In [ ]:
print(df['title'].values.tolist())

In [ ]:
df['title'].values.tolist()